Installing necessary libraries

In [ ]:
!pip install timm

In [ ]:
from timm.data.transforms_factory import create_transform
import timm
from PIL import Image
import timm
import timm.data.mixup as mixup
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from timm.data import create_transform
from timm.utils import ModelEma
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import time, copy

Mounting to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading the pretrained model

In [ ]:
# Define your model
model = timm.create_model('tf_efficientnet_b7', pretrained = True, drop_rate = 0.5, num_classes = 3)

In [ ]:
model.classifier

Linear(in_features=2560, out_features=3, bias=True)

Adding additional linear layers at the end

In [ ]:
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features = 2560, out_features=2560, bias = True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 2560, out_features=2560, bias = True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 2560, out_features=2560, bias = True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 2560, out_features=2560, bias = True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 2560, out_features=2560, bias = True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 2560, out_features=2560, bias = True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 2560, out_features=2560, bias = True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features = 2560, out_features=3, bias = True),

)

In [ ]:
model.classifier

Sequential(
  (0): Linear(in_features=2560, out_features=2560, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2560, out_features=2560, bias=True)
  (3): ReLU()
  (4): Linear(in_features=2560, out_features=2560, bias=True)
  (5): ReLU()
  (6): Linear(in_features=2560, out_features=2560, bias=True)
  (7): ReLU()
  (8): Linear(in_features=2560, out_features=2560, bias=True)
  (9): ReLU()
  (10): Linear(in_features=2560, out_features=2560, bias=True)
  (11): ReLU()
  (12): Linear(in_features=2560, out_features=2560, bias=True)
  (13): ReLU()
  (14): Linear(in_features=2560, out_features=3, bias=True)
)

Getting Image transforms ready

In [ ]:
model.pretrained_cfg

timm.data.resolve_data_config(model.pretrained_cfg)

data_cfg = timm.data.resolve_data_config(model.pretrained_cfg)
transform = timm.data.create_transform(**data_cfg, is_training = True, auto_augment='rand-m9-mstd0.5-inc1')
transform.transforms[0] = transforms.Resize((600, 600))
transform.transforms.pop(1)
transform.transforms[1].ops = transform.transforms[1].ops[:-2]
transform

# transform.transforms[1]

# valid_transform = transforms.Compose([
#         transforms.Resize((600, 600)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ])

# # Load your dataset
# train_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Dog_X_ray/Dog_heart/Train', transform=transform)
# val_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Dog_X_ray/Dog_heart/Valid', transform=valid_transform)

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

Compose(
    Resize(size=(600, 600), interpolation=bilinear, max_size=None, antialias=True)
    RandAugment(n=2, ops=
	AugmentOp(name=AutoContrast, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=Equalize, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=Invert, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=Rotate, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=PosterizeIncreasing, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=SolarizeIncreasing, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=SolarizeAdd, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=ColorIncreasing, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=ContrastIncreasing, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=BrightnessIncreasing, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=SharpnessIncreasing, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=ShearX, p=0.5, m=9, mstd=0.5)
	AugmentOp(name=ShearY, p=0.5, m=9, mstd=0.5))
    MaybeToTensor()
    Normalize(mean=tensor([0.4850, 0.4560, 0.4060]), std=tensor([0.2290, 0.2240, 0.2250]))
)

In [ ]:
# dataloaders = {'Train': train_loader, 'Valid': val_loader}
# dataset_sizes = {'Train': len(train_dataset), 'Valid': len(val_dataset)}
# class_names = train_dataset.classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Unfreezing specific layers

In [ ]:
for param in model.parameters():
  param.requires_grad = False

for param in model.blocks[-3].parameters():
  param.requires_grad = True

for param in model.blocks[-2].parameters():
  param.requires_grad = True

for param in model.blocks[-1].parameters():
  param.requires_grad = True

for param in model.conv_head.parameters():
  param.requires_grad = True

for param in model.classifier.parameters():
  param.requires_grad = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, betas=(0.9, 0.999), eps=1e-8, weight_decay=0.01)
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, threshold=0.0001, threshold_mode='abs')

Training Function

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['Train', 'Valid']:
            if phase == 'Train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)


                    # backward + optimize only if in training phase
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'Train':
                scheduler.step(loss)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'Valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# import copy
# import time
# import os
# import shutil
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader, Subset
# from sklearn.model_selection import KFold
# import torch

# # Define the path to the merged folder
# merged_folder = '/content/drive/MyDrive/Dog_X_ray/Dog_heart/All'

# # Create the merged folder if it doesn't exist
# if not os.path.exists(merged_folder):
#     os.makedirs(merged_folder)

# # Paths to the original folders
# train_folder = '/content/drive/MyDrive/Dog_X_ray/Dog_heart/Train'
# valid_folder = '/content/drive/MyDrive/Dog_X_ray/Dog_heart/Valid'

# # Merge the Train and Valid folders into the All folder
# for folder in [train_folder, valid_folder]:
#     for class_name in os.listdir(folder):
#         class_folder = os.path.join(folder, class_name)
#         merged_class_folder = os.path.join(merged_folder, class_name)

#         if not os.path.exists(merged_class_folder):
#             os.makedirs(merged_class_folder)

#         for file_name in os.listdir(class_folder):
#             file_path = os.path.join(class_folder, file_name)
#             shutil.copy(file_path, merged_class_folder)

In [ ]:
from sklearn.model_selection import KFold
# Define the path to the merged folder
merged_folder = '/content/drive/MyDrive/Dog_X_ray/Dog_heart/All/'

# Load the merged dataset
dataset = datasets.ImageFolder(root=merged_folder, transform=transform)

# Initialize KFold
k_folds = 7
kfold = KFold(n_splits=k_folds, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/model_fold3_7.pth'))

<All keys matched successfully>

In [ ]:
# Perform k-fold cross-validation
for fold, (train_ids, val_ids) in enumerate(kfold.split(dataset)):
    print(f'Fold {fold}')

    # Sample elements randomly from a given list of ids, no replacement
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

    # Define data loaders for training and validation
    train_loader = DataLoader(dataset, batch_size=16, sampler=train_subsampler)
    val_loader = DataLoader(dataset, batch_size=16, sampler=val_subsampler)

    dataloaders = {'Train': train_loader, 'Valid': val_loader}
    dataset_sizes = {'Train': len(train_subsampler), 'Valid': len(val_subsampler)}
    class_names = dataset.classes
    # Train and validate your model
    model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=6)

    # Save the best model weights for this fold
    torch.save(model.state_dict(), f'/content/drive/MyDrive/model_foldlm_{fold}.pth')

Fold 0
Epoch 0/5
----------
Train Loss: 1.0918 Acc: 0.4553
Valid Loss: 0.7346 Acc: 0.5791

Epoch 1/5
----------
Train Loss: 0.8641 Acc: 0.5574
Valid Loss: 0.8069 Acc: 0.4640

Epoch 2/5
----------
Train Loss: 0.9030 Acc: 0.5225
Valid Loss: 1.1124 Acc: 0.5504

Epoch 3/5
----------
Train Loss: 0.9773 Acc: 0.4390
Valid Loss: 1.1123 Acc: 0.4173

Epoch 4/5
----------
Train Loss: 0.8643 Acc: 0.5261
Valid Loss: 1.0093 Acc: 0.4676

Epoch 5/5
----------
Train Loss: 0.8220 Acc: 0.5471
Valid Loss: 0.8774 Acc: 0.4964

Training complete in 21m 53s
Best val Acc: 0.579137
Fold 1
Epoch 0/5
----------
Train Loss: 1.0354 Acc: 0.4420
Valid Loss: 3.7543 Acc: 0.3381

Epoch 1/5
----------
Train Loss: 1.0022 Acc: 0.4456
Valid Loss: 0.9428 Acc: 0.5072

Epoch 2/5
----------
Train Loss: 0.9801 Acc: 0.4709
Valid Loss: 0.9376 Acc: 0.5432

Epoch 3/5
----------
Train Loss: 0.8783 Acc: 0.5177
Valid Loss: 0.7779 Acc: 0.6331

Epoch 4/5
----------
Train Loss: 0.8092 Acc: 0.5604
Valid Loss: 0.8278 Acc: 0.6043

Epoch 5/5


KeyboardInterrupt: 

In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image

model_name = 'model_fold3_7'

model.load_state_dict(torch.load(f'/content/drive/MyDrive/{model_name}.pth'))
model.eval()

# Define a function to make predictions on a single image
def predict_image(image_path):
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((600, 600)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img)
    img_tensor = img_tensor.unsqueeze(0)
    img_tensor = img_tensor.cuda()
    output = model(img_tensor)
    _, predicted = torch.max(output, 1)
    return predicted.item()

paths = []
answers = []
image_folder = '/content/drive/MyDrive/Dog_X_ray/Test/'
filenames = os.listdir(image_folder)
for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    prediction = predict_image(image_path)

    paths.append(image_path)
    answers.append(prediction)

import pandas as pd
df = pd.DataFrame({filenames[0]: filenames[1:], answers[0]: answers[1:]})
df

df.to_csv(f'/content/drive/MyDrive/{model_name}.csv', index=False)